<a href="https://colab.research.google.com/github/cmlins/fast_track/blob/main/tree/main/Engenharia%2520de%2520dados/desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio 2

## Doações Eleições 2014

### Instalações e imports das bibliotecas, início da sessão spark

In [1]:
# Instala bibliotecas necessárias

!pip install --upgrade plotly
!pip install pyspark

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


In [2]:
# import das bibliotecas usadas

import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import isnan, when, count, col, isnull, asc, desc, mean, sum, lit, format_string, concat

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
# Cria uma sessão spark
spark = SparkSession.builder.master('local').appName('DataWrangling').getOrCreate()

# Configura para que a visualização das saídas seja similar ao Pandas
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

sc = spark.sparkContext

In [4]:
#montar o drive
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


### Ranking dos 10 partidos que mais receberam doações em 2014

In [5]:
end_candidatos = '/drive/MyDrive/Desafio/receitas_candidatos_2014_brasil.txt'
end_comites ='/drive/MyDrive/Desafio/receitas_comites_2014_brasil.txt'
end_partidos = '/drive/MyDrive/Desafio/receitas_partidos_2014_brasil.txt'

In [6]:
def carrega_df(endereco):
  return spark.read.option("header", "true") \
    .option("delimiter", ";") \
    .option("inferSchema", "true") \
    .option("encoding", "ISO-8859-1") \
    .csv(endereco)

### Doações a Candidatos


In [7]:
candidatos = carrega_df(end_candidatos)
candidatos.limit(5)

Cód. Eleição,Desc. Eleição,Data e hora,CNPJ Prestador Conta,Sequencial Candidato,UF,Sigla Partido,Numero candidato,Cargo,Nome candidato,CPF do candidato,Numero Recibo Eleitoral,Numero do documento,CPF/CNPJ do doador,Nome do doador,Nome do doador (Receita Federal),Sigla UE doador,Número partido doador,Número candidato doador,Cod setor econômico do doador,Setor econômico do doador,Data da receita,Valor receita,Tipo receita,Fonte recurso,Especie recurso,Descricao da receita,CPF/CNPJ do doador originário,Nome do doador originário,Tipo doador originário,Setor econômico do doador originário,Nome do doador originário (Receita Federal)
143,Eleições Gerais 2014,09/07/201617:15:01,20578194000114,90000000637,GO,PSDB,45,Governador,MARCONI FERREIRA ...,3553821809,000450300000GO000064,20141550000,01256007000131,GOVESA GOIANIA VE...,GOVESA GOIANIA VE...,#NULO,#NULO,45,4511101,Comércio a varejo...,02/10/201400:00:00,145000,Recursos de pesso...,Nao especificado,Transferência ele...,#NULO,#NULO,#NULO,#NULO,#NULO,#NULO
143,Eleições Gerais 2014,09/07/201617:15:01,20571797000194,90000000725,GO,PSD,5555,Deputado Federal,JOSE MARIO SCHREINER,41877004987,055550600000GO000090,553338000003336,01256007000131,GOVESA GOIANIA VE...,GOVESA GOIANIA VE...,#NULO,#NULO,5555,4511101,Comércio a varejo...,16/10/201400:00:00,100000,Recursos de pesso...,Nao especificado,Transferência ele...,#NULO,#NULO,#NULO,#NULO,#NULO,#NULO
143,Eleições Gerais 2014,09/07/201617:15:01,20578262000145,180000000046,PI,PSB,400,Senador,WILSON NUNES MARTINS,6444555353,004000500000PI000086,#NULO,20574446000137,ELEICAO 2014 GUST...,ELEICAO 2014 GUST...,PI,40,400,9492800,Atividades de org...,18/07/201400:00:00,725,Recursos de outro...,Outros Recursos n...,Estimado,CARTAZ F2 5000 UN...,39817806391,GUSTAVO SOUSA DE ...,F,#NULO,GUSTAVO SOUSA DE ...
143,Eleições Gerais 2014,09/07/201617:15:01,20603227000139,180000000044,PI,PMDB,15,Governador,ANTONIO JOSE DE M...,27361136320,000150300000PI000035,#NULO,20574446000137,ELEIÇAO GUSTAVO S...,ELEICAO 2014 GUST...,PI,40,15,9492800,Atividades de org...,23/07/201400:00:00,1875,Recursos de outro...,Outros Recursos n...,Estimado,ADESIVOS NF 607 D...,39817806391,GUSTAVO DE SOUSA ...,F,#NULO,GUSTAVO SOUSA DE ...
143,Eleições Gerais 2014,09/07/201617:15:01,20603227000139,180000000044,PI,PMDB,15,Governador,ANTONIO JOSE DE M...,27361136320,000150300000PI000034,#NULO,20574446000137,ELEIÇAO GUSTAVO S...,ELEICAO 2014 GUST...,PI,40,15,9492800,Atividades de org...,18/07/201400:00:00,725,Recursos de outro...,Outros Recursos n...,Estimado,CARTAZ F2 GOV ZE ...,39817806391,GUSTAVO DE SOUSA ...,F,#NULO,GUSTAVO SOUSA DE ...


### Doações a Comitês

In [8]:
comites = carrega_df(end_comites)
comites.limit(5)

Cód. Eleição,Desc. Eleição,Data e hora,CNPJ Prestador Conta,Sequencial Comite,UF,Tipo Comite,Sigla Partido,Tipo do documento,Número do documento,CPF/CNPJ do doador,Nome do doador,Nome do doador (Receita Federal),Sigla UE doador,Número partido doador,Número candidato doador,Cod setor econômico do doador,Setor econômico do doador,Data da receita,Valor receita,Tipo receita,Fonte recurso,Espécie recurso,Descrição da receita,CPF/CNPJ do doador originário,Nome do doador originário,Tipo doador originário,Setor econômico do doador originário,Nome do doador originário (Receita Federal)
143,Eleições Gerais 2014,09/07/2016 17:24:00,20603433000149,448543,RS,Comitê Financeiro...,PMDB,C15000588013RS000439,2164,88348024000187,SBS ENGENHARIA E ...,SBS ENGENHARIA E ...,#NULO,#NULO,#NULO,4211101,Construção de rod...,29/10/201400:00:00,50000,Recursos de pesso...,Nao especificado,Transferência ele...,#NULO,#NULO,#NULO,#NULO,#NULO,#NULO
143,Eleições Gerais 2014,09/07/2016 17:24:00,20603433000149,448543,RS,Comitê Financeiro...,PMDB,C15000588013RS000375,2385,02955426000124,CASTELLAR ENGENHARIA,CASTELLAR ENGENHA...,#NULO,#NULO,#NULO,4211101,Construção de rod...,10/09/201400:00:00,20000,Recursos de pesso...,Nao especificado,Transferência ele...,#NULO,#NULO,#NULO,#NULO,#NULO,#NULO
143,Eleições Gerais 2014,09/07/2016 17:24:00,20632466000117,785026,RJ,Comitê Financeiro...,PMDB,C15000560011RJ000046,000033,09446911000111,CPM CONSTRUÇÕES LTDA,C P M CONSTRUCOES...,#NULO,#NULO,#NULO,4211101,Construção de rod...,03/09/201400:00:00,100000,Recursos de pesso...,Nao especificado,Transferência ele...,#NULO,#NULO,#NULO,#NULO,#NULO,#NULO
143,Eleições Gerais 2014,09/07/2016 17:24:00,20609045000175,615325,MG,Comitê Financeiro...,PT,C13000541238MG000018,000011,66289471000170,LIBE CONSTRUTORA ...,LIBE CONSTRUTORA ...,#NULO,#NULO,#NULO,4211101,Construção de rod...,07/10/201400:00:00,100000,Recursos de pesso...,Nao especificado,Transferência ele...,#NULO,#NULO,#NULO,#NULO,#NULO,#NULO
143,Eleições Gerais 2014,09/07/2016 17:24:00,20609045000175,615325,MG,Comitê Financeiro...,PT,C13000541238MG000019,999467753000011,66289471000170,LIBE CONSTRUTORA ...,LIBE CONSTRUTORA ...,#NULO,#NULO,#NULO,4211101,Construção de rod...,14/10/201400:00:00,100000,Recursos de pesso...,Nao especificado,Transferência ele...,#NULO,#NULO,#NULO,#NULO,#NULO,#NULO


### Doações a Partidos

In [9]:
partidos = carrega_df(end_partidos)
partidos.limit(5)

Cód. Eleição,Desc. Eleição,Data e hora,CNPJ Prestador Conta,Sequencial Diretorio,UF,Tipo diretorio,Sigla Partido,Tipo do documento,Número do documento,CPF/CNPJ do doador,Nome do doador,Nome do doador (Receita Federal),Sigla UE doador,Número partido doador,Número candidato doador,Cod setor econômico do doador,Setor econômico do doador,Data da receita,Valor receita,Tipo receita,Fonte recurso,Espécie recurso,Descrição da receita,CPF/CNPJ do doador originário,Nome do doador originário,Tipo doador originário,Setor econômico do doador originário,Nome do doador originário (Receita Federal)
143,Eleições Gerais 2014,09/07/2016 17:23:04,59941682000180,41682,SP,Direção Estadual/...,PSDB,P45000371072SP000182,662945000452014,03653474000120,Direção Nacional,PARTIDO DA SOCIAL...,BR,45,#NULO,9492800,Atividades de org...,25-SEP-14,50000,Recursos de parti...,Outros Recursos n...,Transferência ele...,#NULO,51990695000137,BRADESCO VIDA E P...,J,Previdência compl...,BRADESCO VIDA E P...
143,Eleições Gerais 2014,09/07/2016 17:23:04,24489114000105,23874,PB,Direção Estadual/...,PSDB,P45000320516PB000013,000,03653474000120,Direção Nacional,PARTIDO DA SOCIAL...,BR,45,#NULO,9492800,Atividades de org...,06-AUG-14,500000,Recursos de parti...,Outros Recursos n...,Transferência ele...,#NULO,51990695000137,BRADESCO VIDA E P...,J,Previdência compl...,BRADESCO VIDA E P...
143,Eleições Gerais 2014,09/07/2016 17:23:04,14781335000107,30210,RS,Direção Estadual/...,PRTB,P28000388013RS000005,25310402004,74703034000189,Direção Estadual/...,PARTIDO PROGRESSISTA,RS,11,#NULO,9492800,Atividades de org...,08-SEP-14,10000,Recursos de parti...,Outros Recursos n...,Depósito em espécie,#NULO,51990695000137,BRADESCO VIDA E P...,J,Previdência compl...,BRADESCO VIDA E P...
143,Eleições Gerais 2014,09/07/2016 17:23:04,14781335000107,30210,RS,Direção Estadual/...,PRTB,P28000388013RS000003,000057,74703034000189,Direção Estadual/...,PARTIDO PROGRESSISTA,RS,11,#NULO,9492800,Atividades de org...,19-AUG-14,10000,Recursos de parti...,Outros Recursos n...,Cheque,#NULO,51990695000137,BRADESCO VIDA E P...,J,Previdência compl...,BRADESCO VIDA E P...
143,Eleições Gerais 2014,09/07/2016 17:23:04,3653474000120,234,BR,Direção Nacional,PSDB,P45000200000BR000331,128763,20572776000193,AÉCIO NEVES DA CUNHA,ELEICAO 2014 AECI...,BR,45,#NULO,9492800,Atividades de org...,17-OCT-14,2000000,Recursos de outro...,Outros Recursos n...,Transferência ele...,#NULO,51990695000137,BRADESCO VIDA E P...,J,Previdência compl...,BRADESCO VIDA E P...


In [10]:
doacoes_partidos = partidos.groupBy('Sigla  Partido').agg(sum('Valor receita'))
doacoes_partidos = doacoes_partidos.withColumnRenamed('sum(Valor receita)','receita_partidos').withColumnRenamed('Sigla  Partido','sigla_partido')
# doacoes_partidos

In [11]:
doacoes_candidatos = candidatos.groupBy('Sigla  Partido').agg(sum('Valor receita'))
doacoes_candidatos = doacoes_candidatos.withColumnRenamed('sum(Valor receita)','receita_candidatos').withColumnRenamed('Sigla  Partido','candidatos_partidos')
# doacoes_candidatos

In [12]:
doacoes_comites = comites.groupBy('Sigla  Partido').agg(sum('Valor receita'))
doacoes_comites = doacoes_comites.withColumnRenamed('sum(Valor receita)','receita_comites').withColumnRenamed('Sigla  Partido','comites_partidos')
# doacoes_comites

In [13]:
doacoes = doacoes_partidos.join(doacoes_candidatos, doacoes_partidos.sigla_partido == doacoes_candidatos.candidatos_partidos)
doacoes2 = doacoes.join(doacoes_comites, doacoes.sigla_partido == doacoes_comites.comites_partidos)

# doacoes2

In [14]:
# doacoes2['total_doacoes'] = doacoes2['receita_partidos'] + doacoes2['receita_candidatos'] + doacoes2['receita_comites']
# donations = doacoes2.withColumn("total_doacoes", doacoes2('receita_partidos') + doacoes2('receita_candidatos') + doacoes2('receita_comites'))
total_receita = doacoes2.withColumn("total_doacoes", doacoes2.receita_partidos + doacoes2.receita_candidatos + doacoes2.receita_comites)

In [15]:
total_receita = total_receita.sort(desc('total_doacoes')).limit(10)

In [16]:
total_receita = total_receita.drop('candidatos_partidos','comites_partidos').sort(desc('total_doacoes'))

### 10 maiores receitas de Partidos

In [17]:
total_receita.show()

+-------------+----------------+------------------+---------------+-------------+
|sigla_partido|receita_partidos|receita_candidatos|receita_comites|total_doacoes|
+-------------+----------------+------------------+---------------+-------------+
|           PT|    3.16596668E8|      9.19956868E8|     7.274574E7|1.309299276E9|
|         PSDB|    3.33840403E8|      6.75281978E8|   2.93740588E8|1.302862969E9|
|         PMDB|    4.68168731E8|      6.38647407E8|   1.49043725E8|1.255859863E9|
|          PSB|     8.9395508E7|      2.83854798E8|    9.5294402E7| 4.68544708E8|
|          PSD|    1.29086966E8|      2.31207726E8|      1621380.0| 3.61916072E8|
|           PP|     1.2224843E8|      1.86493644E8|      7357800.0| 3.16099874E8|
|           PR|     9.9068421E7|      1.39827438E8|      6319902.0| 2.45215761E8|
|          DEM|     8.5227612E7|      1.47902837E8|      3910856.0| 2.37041305E8|
|          PTB|     5.3939504E7|      1.24983295E8|      4727545.0| 1.83650344E8|
|          PDT| 

In [18]:
fig = px.bar(total_receita.toPandas(), x='sigla_partido', y='total_doacoes',
             labels={'total_doacoes': 'receita', 'sigla_partido': ''})
# fig.update_yaxes(type="log")
fig.show()

### A votação que os candidatos a presidente tiveram no segundo turno e quanto seus partidos arrecadaram

In [19]:
candidatos = spark.read.csv('/drive/MyDrive/Desafio/receitas_candidatos_2014_brasil_english.csv', header=True)
# candidatos.limit(5)
candidatos = candidatos.select('cat_candidate_name', 'id_candidate_num', 'cat_party', 'cat_political_office')
presidentes = candidatos.filter(candidatos.cat_political_office == 'Presidente')
presidentes = presidentes.dropDuplicates()
presidentes = presidentes.withColumnRenamed('cat_candidate_name','cat_candidate_name_pres').withColumnRenamed('id_candidate_num','id_candidate_num_pres').withColumnRenamed('cat_party','cat_party_pres').withColumnRenamed('cat_political_office','cat_political_office_pres')

# presidentes.show()

In [20]:
votacao = spark.read.csv('/drive/MyDrive/Desafio/votacao_brasil_english.csv',header=True)
votacao = votacao.filter(votacao.num_turn == 2)

In [21]:
presidentes_segundo_turno = votacao.join(presidentes, votacao.cat_candidate_name == presidentes.cat_candidate_name_pres)

In [23]:
presidentes_segundo_turno = presidentes_segundo_turno.groupBy('cat_candidate_name',).agg(sum('num_votes'))

In [24]:
presidentes_segundo_turno = presidentes_segundo_turno.withColumnRenamed('cat_candidate_name','candidato').withColumnRenamed('sum(num_votes)','total_votos')
presidentes_segundo_turno = presidentes_segundo_turno.join(presidentes, presidentes.cat_candidate_name_pres == presidentes_segundo_turno.candidato)
presidentes_segundo_turno = presidentes_segundo_turno.join(total_receita, presidentes_segundo_turno.cat_party_pres == total_receita.sigla_partido)
presidentes_segundo_turno = presidentes_segundo_turno.drop('cat_political_office_pres', 'cat_candidate_name_pres','sigla_partido')
presidentes_segundo_turno = presidentes_segundo_turno.withColumn("candidato_partido", concat(col('id_candidate_num_pres'), lit(' - '), col('candidato')))

# presidentes_segundo_turno

In [29]:
# converte colunas em uma lista
labels = presidentes_segundo_turno.select('candidato_partido').toPandas()['candidato_partido'].tolist()
total_votos = presidentes_segundo_turno.select('total_votos').toPandas()['total_votos'].tolist()
total_doacoes = presidentes_segundo_turno.select('total_doacoes').toPandas()['total_doacoes'].tolist()

# plota subgráficos com as colunas 'total_doacoes' e 'total_votos'
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], subplot_titles=['receita', 'votos'])
fig.add_trace(go.Pie(labels=labels, values=total_doacoes, name="receita"), 
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=total_votos, name="votos"),
              1, 2)

fig.update_layout(height=600, width=1200, title_text='Receita X Votação')